In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import atlite
import rioxarray
import xarray as xr

In [ ]:
tz=(pd.read_csv(snakemake.input.time_zones)
        .loc[:,["ISO2","UTC"]])

#year=snakemake.wildcards.year

#degree_day_years=snakemake.params.degree_day_years

weather_data=snakemake.input.weather_data_path
pop_data=snakemake.input.pop_raster
onshore_shape=snakemake.input.onshore_shape
yr=snakemake.wildcards.degree_day_year

In [ ]:
pop = rioxarray.open_rasterio(pop_data, masked=True)

In [ ]:
europe=(gpd.read_file(onshore_shape)
    .dissolve(by="CNTR_CODE")
    .drop(columns="index")
    .rename(index={"UK":"GB"}))

In [ ]:
out=[]

ds = xr.open_dataset(weather_data+"\\europe_"+str(yr)+".nc", chunks="auto")

cutout = atlite.Cutout(
path="tmp",
data=ds
)
for z in europe.index:
    print(z)
    
    geo=europe.reset_index().query("CNTR_CODE == @z")

    ztz=tz.loc[tz["ISO2"]==z,"UTC"].values[0]

    cutout1=cutout.sel(
        x=slice(geo.bounds["minx"].values[0]-1,geo.bounds["maxx"].values[0]+1),
        y=slice(geo.bounds["miny"].values[0]-1,geo.bounds["maxy"].values[0]+1)
        )

    geo=geo.to_crs(pop.rio.crs)

    layout = (
    pop.rio.clip(
        geo.geometry
    ) 
    .rio.reproject( 
        cutout1.crs,
        shape=cutout1.shape,
        transform=cutout1.transform,
        resampling=13,
    )
    .squeeze()
    )

    layout = layout / layout.sum()

    t2m=cutout1.temperature(layout=layout.fillna(0.0)).squeeze().to_pandas()

    # in original code we take this "mean" temp approach to try and capture building interia (I think).
    # commented out because I can't remember where it came from.
    #t2m=0.5*t2m+0.5*t2m.shift(-1)

    cdh=t2m-20
    cdh[cdh < 0.]=0.0
    
    hdh=pd.Series(np.where((t2m.index.hour >=7-ztz) & (t2m.index.hour <=22-ztz),15-t2m,10-t2m),index=t2m.index)
    hdh[hdh < 0.]=0.0

    df=pd.concat((t2m.rename(z),cdh.rename(z+"_cdh"),hdh.rename(z+"_hdh")),axis=1)
    out.append(df)
        
out=pd.concat(out,axis=1).reset_index().rename({"time":"UTC"},axis=1)
out.to_csv(snakemake.output[0],index=False,float_format='%.3f')